In [1]:
import os
# os.chdir("..")
# os.getcwd()

In [3]:
import pandas as pd
data_path = "ma_2021"
mol_df = pd.read_csv(os.path.join(data_path,"molecules.csv"))
mol_df.head()

,CID,MolecularWeight,IsomericSMILES,IUPACName,name
0,179,88.11,CC(C(=O)C)O,3-hydroxybutan-2-one,acetoin
1,240,106.12,C1=CC=C(C=C1)C=O,benzaldehyde,benzaldehyde
2,261,72.11,CCCC=O,butanal,butyraldehyde
3,454,128.21,CCCCCCCC=O,octanal,octanal
4,650,86.09,CC(=O)C(=O)C,"butane-2,3-dione","2,3-butanedione"


In [8]:
len(mol_df)

72

In [9]:
mol_df["MFP"] = mol_df["IsomericSMILES"]

In [10]:
cid_to_embed = mol_df[["CID","MFP"]].set_index("CID").to_dict()["MFP"]
len(cid_to_embed), next(iter(cid_to_embed.items()))

(72, (179, 'CC(C(=O)C)O'))

In [5]:
blends_df = pd.read_csv(os.path.join(data_path,"behavior.csv"))
blends_df

,Stimulus A,Stimulus B,Subject,Rep,IA,IAmix,IB,IBmix,IAB,PA,PB,PAB
0,179,31249,2,1,1.6,0.0,5.6,7.0,6.8,6.3,5.6,5.5
1,179,31249,3,1,5.0,5.3,4.9,5.1,6.3,6.8,3.6,4.2
2,179,31249,5,1,5.2,4.7,5.4,3.8,4.9,3.7,5.3,3.8
3,179,31249,7,1,4.1,0.0,8.0,10.0,7.0,4.0,6.0,6.0
4,179,31249,14,1,5.2,5.7,5.8,4.1,5.4,5.1,5.1,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...
6655,9855795,5281167,56,1,5.0,7.0,4.0,5.6,9.0,4.0,4.0,0.7
6656,9855795,5281167,59,1,8.7,9.6,8.5,11.0,9.0,2.0,2.9,1.8
6657,9855795,5281167,60,1,5.2,3.9,5.5,3.0,5.1,8.0,7.8,7.1
6658,9855795,5281167,62,1,5.1,2.6,3.8,0.0,5.2,8.0,9.3,9.5


In [6]:
all_blends = set()
for i, row in blends_df.iterrows():
    all_blends.add((row["Stimulus A"],row["Stimulus B"]))
print(f"Found {len(all_blends)} unique blends.")

Found 198 unique blends.


In [15]:
import numpy as np

# Stack the prediction targets
value_cols = ["IA", "IAmix", "IB", "IBmix", "IAB", "PA", "PB", "PAB"]
blends_df["Values"] = blends_df.apply(lambda row: np.stack(row[value_cols]).tolist(), axis=1)

group_df = blends_df.groupby(["Stimulus A", "Stimulus B"])

# Calculate the mean for the "Values" column along axis=0
result_df = group_df["Values"].apply(
    lambda group: np.mean(np.stack(group.to_numpy()), axis=0).tolist()
).reset_index(name="Values")

# Display the resulting DataFrame
result_df.head()

,Stimulus A,Stimulus B,Values
0,179,31249,"[5.623333333333333, 5.656666666666667, 6.30666..."
1,240,61592,"[7.280000000000001, 5.889999999999999, 4.28666..."
2,261,61592,"[6.826666666666665, 6.336666666666667, 4.93666..."
3,454,7590,"[5.993333333333331, 4.973333333333333, 6.22666..."
4,454,31260,"[5.790000000000001, 4.300000000000001, 5.55999..."


In [16]:
import numpy as np

# Make MFP emeddings for each molecule
result_df["SMILES_A"] = result_df["Stimulus A"].map(cid_to_embed)
result_df["SMILES_B"] = result_df["Stimulus B"].map(cid_to_embed)

result_df

,Stimulus A,Stimulus B,Values,SMILES_A,SMILES_B
0,179,31249,"[5.623333333333333, 5.656666666666667, 6.30666...",CC(C(=O)C)O,CCOC(=O)CCC(=O)OCC
1,240,61592,"[7.280000000000001, 5.889999999999999, 4.28666...",C1=CC=C(C=C1)C=O,CCOC(=O)CCSC
2,261,61592,"[6.826666666666665, 6.336666666666667, 4.93666...",CCCC=O,CCOC(=O)CCSC
3,454,7590,"[5.993333333333331, 4.973333333333333, 6.22666...",CCCCCCCC=O,CCOC(=O)CC1=CC=CC=C1
4,454,31260,"[5.790000000000001, 4.300000000000001, 5.55999...",CCCCCCCC=O,CC(C)CCO
...,...,...,...,...,...
193,9855795,7463,"[6.296666666666667, 5.696666666666666, 5.84666...",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CC1=CC=C(C=C1)C(C)C
194,9855795,7765,"[6.203333333333333, 5.500000000000001, 5.17833...",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CCOC(C)OCC
195,9855795,8093,"[6.319999999999999, 5.5600000000000005, 6.2933...",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CCCCCCC(=O)C
196,9855795,31249,"[6.0, 4.873333333333334, 5.833333333333334, 4....",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CCOC(=O)CCC(=O)OCC


In [13]:
from rdkit import Chem

def canonical_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles = True)

def remove_stereo(smiles):
    try:
        _smiles = smiles.replace('@','')
        _smiles = _smiles.replace('/','')
        _smiles = _smiles.replace('\\','')
        return canonical_smiles(_smiles)
    except Exception as e:
        raise Exception(f"error for smiles {smiles}")


def join_smiles(smiles_list, _remove_stereo = False):
    try:
        smi_1 = smiles_list[0]
        smi_2 = smiles_list[1]
        if smi_1[:5] =="InChI":
            smi_1 = Chem.MolToSmiles(Chem.MolFromInchi(smi_1))
        if smi_2[:5] =="InChI":
            smi_2 = Chem.MolToSmiles(Chem.MolFromInchi(smi_2))
        if _remove_stereo:
            mol_1 = Chem.MolFromSmiles(remove_stereo(smi_1))
            mol_2 = Chem.MolFromSmiles(remove_stereo(smi_2))
        else:
            mol_1 = Chem.MolFromSmiles(smi_1)
            mol_2 = Chem.MolFromSmiles(smi_2)
        combo = Chem.CombineMols(mol_1,mol_2)
        comb_smi = Chem.MolToSmiles(combo)
        return comb_smi
    except Exception as e:
        print(e)
        return None

In [17]:
result_df['combined_SMILES_canon_non_stereo'] = result_df.apply(lambda x: join_smiles([x.SMILES_A, x.SMILES_B], _remove_stereo = True), axis=1)

In [18]:
result_df.head()

,Stimulus A,Stimulus B,Values,SMILES_A,SMILES_B,combined_SMILES_canon_non_stereo
0,179,31249,"[5.623333333333333, 5.656666666666667, 6.30666...",CC(C(=O)C)O,CCOC(=O)CCC(=O)OCC,CC(=O)C(C)O.CCOC(=O)CCC(=O)OCC
1,240,61592,"[7.280000000000001, 5.889999999999999, 4.28666...",C1=CC=C(C=C1)C=O,CCOC(=O)CCSC,CCOC(=O)CCSC.O=Cc1ccccc1
2,261,61592,"[6.826666666666665, 6.336666666666667, 4.93666...",CCCC=O,CCOC(=O)CCSC,CCCC=O.CCOC(=O)CCSC
3,454,7590,"[5.993333333333331, 4.973333333333333, 6.22666...",CCCCCCCC=O,CCOC(=O)CC1=CC=CC=C1,CCCCCCCC=O.CCOC(=O)Cc1ccccc1
4,454,31260,"[5.790000000000001, 4.300000000000001, 5.55999...",CCCCCCCC=O,CC(C)CCO,CC(C)CCO.CCCCCCCC=O


load model (mpnn)

In [ ]:
# !pip install openpom

In [20]:
import deepchem as dc
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio
from openpom.models.mpnn_pom import MPNNPOMModel

In [21]:
TASKS = ['acidic', 'aldehydic', 'alliaceous', 'amber', 'animal', 'anise', 'aromatic',
         'balsamic', 'berry', 'bitter', 'bready', 'brown', 'burnt', 'buttery',
         'camphoreous','caramellic','cheesy','chemical','chocolate','citrus','clean',
         'cocoa','coconut','coffee','cooling','coumarinic','creamy', 'dairy', 'earthy',
         'estery', 'ethereal', 'fatty', 'fermented', 'floral', 'fresh', 'fruity', 'fungal',
         'fusel', 'green', 'herbal', 'honey', 'jammy', 'licorice', 'marine', 'meaty',
         'medicinal','melon','mentholic','minty','mossy','musk','musty','nutty','oily',
         'onion', 'orris', 'phenolic', 'powdery', 'roasted', 'rummy', 'soapy', 'solvent',
         'sour', 'spicy', 'sulfurous', 'sweet', 'thujonic', 'tonka', 'tropical', 'vanilla',
         'vegetable', 'waxy', 'winey', 'woody']
print("No of tasks: ", len(TASKS))

No of tasks:  74


In [22]:
input_file = 'mix_50_train_split.csv'

In [23]:
# get dataset

featurizer = GraphFeaturizer()
# smiles_field = 'combined_smiles_non_stereo'
# loader = dc.data.CSVLoader(tasks=TASKS,
#                    feature_field=smiles_field,
#                    featurizer=featurizer)
# dataset = loader.create_dataset(inputs=[input_file], data_dir="./datasets/train_mix")
# n_tasks = len(dataset.tasks)

train_dataset = dc.data.DiskDataset(data_dir="/media/ary2260/USBHD/Odor_mix_research/odor-mix/odor_pair_check_single_fold/datasets/train_mix")

In [24]:
# smiles_field = 'combined_smiles_non_stereo'
# loader = dc.data.CSVLoader(tasks=TASKS,
#                    feature_field=smiles_field,
#                    featurizer=featurizer)
# test_dataset = loader.create_dataset(inputs=["mix_50_test_split.csv"], data_dir="./datasets/test_mix/")
# n_tasks = len(test_dataset.tasks)

test_dataset = dc.data.DiskDataset(data_dir="/media/ary2260/USBHD/Odor_mix_research/odor-mix/odor_pair_check_single_fold/datasets/test_mix")

In [25]:
n_tasks = len(test_dataset.tasks)

In [26]:
len(train_dataset)

41612

In [27]:
len(test_dataset)

37932

In [28]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

In [29]:
learning_rate = dc.models.optimizers.ExponentialDecay(initial_rate=0.001, decay_rate=0.5, decay_steps=840, staircase=True)
# learning_rate = 9.4e-3
# learning_rate_scale = 1

In [30]:
# initialize model

model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=128,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 100,
                            edge_hidden_feats = 75,
                            edge_out_feats = 100,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'sum',
                            mode = 'classification',
                            number_atom_features = GraphConvConstants.ATOM_FDIM,
                            number_bond_features = GraphConvConstants.BOND_FDIM,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 3,
                            ffn_hidden_list= [300],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.12,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1e-05,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 100,
                            model_dir = './examples/experiments',
                            device_name='cuda')

In [31]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)


In [32]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [33]:
model.restore("/media/ary2260/USBHD/Odor_mix_research/odor-mix/odor_pair_check_single_fold/odor_experiment_model_ckp/results/exp2_odor_mix_3_trials/trial_count_trial_2_2024-01-31 14_05_54.206921/best_ckpt_trial_2_1.pt")

INFO:deepchem.models.torch_models.torch_model:Restoring model


visualize results

In [40]:
def get_embed(smiles):
    X = featurizer.featurize(smiles)
    embed = model.predict_embedding(dc.data.NumpyDataset(X=X))
    return embed.reshape(-1)

In [35]:
result_df.columns

Index(['Stimulus A', 'Stimulus B', 'Values', 'SMILES_A', 'SMILES_B',
       'combined_SMILES_canon_non_stereo'],
      dtype='object')

In [ ]:
result_df['Embed'] = result_df['combined_SMILES_canon_non_stereo'].apply(lambda x: get_embed(x))

In [43]:
result_df

,Stimulus A,Stimulus B,Values,SMILES_A,SMILES_B,combined_SMILES_canon_non_stereo,Embed
0,179,31249,"[5.623333333333333, 5.656666666666667, 6.30666...",CC(C(=O)C)O,CCOC(=O)CCC(=O)OCC,CC(=O)C(C)O.CCOC(=O)CCC(=O)OCC,"[-1.1569222, -0.33323765, -0.33564734, 0.22936..."
1,240,61592,"[7.280000000000001, 5.889999999999999, 4.28666...",C1=CC=C(C=C1)C=O,CCOC(=O)CCSC,CCOC(=O)CCSC.O=Cc1ccccc1,"[-0.77010393, -0.49282017, -0.30407417, -0.053..."
2,261,61592,"[6.826666666666665, 6.336666666666667, 4.93666...",CCCC=O,CCOC(=O)CCSC,CCCC=O.CCOC(=O)CCSC,"[-0.51542854, -0.38390723, -0.64058393, -0.185..."
3,454,7590,"[5.993333333333331, 4.973333333333333, 6.22666...",CCCCCCCC=O,CCOC(=O)CC1=CC=CC=C1,CCCCCCCC=O.CCOC(=O)Cc1ccccc1,"[-0.82888156, -0.3095377, -0.849676, 0.0401058..."
4,454,31260,"[5.790000000000001, 4.300000000000001, 5.55999...",CCCCCCCC=O,CC(C)CCO,CC(C)CCO.CCCCCCCC=O,"[-1.1116099, -0.19436063, -0.96364623, 0.46847..."
...,...,...,...,...,...,...,...
193,9855795,7463,"[6.296666666666667, 5.696666666666666, 5.84666...",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CC1=CC=C(C=C1)C(C)C,C=C(C)C1CCC2=CCCC(C)C2(C)C1.Cc1ccc(C(C)C)cc1,"[-0.85202134, 0.38667628, -0.5630481, -0.24532..."
194,9855795,7765,"[6.203333333333333, 5.500000000000001, 5.17833...",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CCOC(C)OCC,C=C(C)C1CCC2=CCCC(C)C2(C)C1.CCOC(C)OCC,"[-1.0058557, 0.51016283, -0.49984628, 0.228909..."
195,9855795,8093,"[6.319999999999999, 5.5600000000000005, 6.2933...",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CCCCCCC(=O)C,C=C(C)C1CCC2=CCCC(C)C2(C)C1.CCCCCCC(C)=O,"[-0.8618385, 0.051085044, -0.7161786, -0.18003..."
196,9855795,31249,"[6.0, 4.873333333333334, 5.833333333333334, 4....",C[C@@H]1CCC=C2[C@]1(C[C@@H](CC2)C(=C)C)C,CCOC(=O)CCC(=O)OCC,C=C(C)C1CCC2=CCCC(C)C2(C)C1.CCOC(=O)CCC(=O)OCC,"[-0.6691275, 0.5423545, -0.10363586, 0.7696569..."


end

In [44]:
import scipy.stats as stats
import sklearn
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import sklearn.multioutput
from tqdm.notebook import tqdm
import numpy as np

n_splits = 5

def train_model(df):
    # Convert "Embed" and "Values" to matrices
    X = np.vstack(df["Embed"].to_numpy())  # Convert list/array-like embeddings to a matrix
    y = np.vstack(df["Values"].to_numpy())

    # Initialize k-fold cross-validation
    kfold = sklearn.model_selection.KFold(n_splits=n_splits, shuffle=False)

    # Initialize a dictionary to store MSEs for each value
    mse_dict = {col: [] for col in value_cols}

    for train_index, test_index in tqdm(kfold.split(X), total=n_splits):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = sklearn.multioutput.MultiOutputRegressor(SVR())
        model.fit(X_train, y_train)

        # Predict the entire vector
        y_pred = model.predict(X_test)

        # Calculate and store MSE for each value (column)
        for i, col in enumerate(value_cols):
            mse = mean_squared_error(y_test[:, i], y_pred[:, i])
            mse_dict[col].append(mse)

    results_dict = dict()        
    # Report MSE for each fold and value with mean ± confidence interval
    for col, mse_list in mse_dict.items():
        # Calculate mean and 95% confidence interval
        average_mse = np.mean(mse_list)
        std_error = stats.sem(mse_list)  # Standard error of the mean
        confidence_interval = stats.t.interval(
            0.95, len(mse_list) - 1, loc=average_mse, scale=std_error
        )

        lower_bound, upper_bound = confidence_interval
        result = f"{average_mse:.4f} ± {(upper_bound - lower_bound) / 2:.4f}"
        results_dict[col] = average_mse
    return results_dict


In [45]:
mpnn_results = train_model(result_df)
mpnn_results

  0%|          | 0/5 [00:00<?, ?it/s]

{'IA': 0.5680863508190361,
 'IAmix': 0.9571730548186119,
 'IB': 0.590631943989745,
 'IBmix': 0.8998285769654653,
 'IAB': 0.2314512882054316,
 'PA': 0.8548348100941959,
 'PB': 0.8826446861243309,
 'PAB': 0.813826868877125}

In [18]:
labels = ["MFP","GIN-GNN","GIN-LOGITS"]
all_df = pd.DataFrame([mfp_results, gin_results, logits_results],index=labels)
all_df

,IA,IAmix,IB,IBmix,IAB,PA,PB,PAB
MFP,0.484053,0.878455,0.403802,0.705960,0.227715,0.696112,0.438578,0.704216
GIN-GNN,0.512021,0.839607,0.599644,0.906537,0.262168,0.847067,0.851727,0.895357
GIN-LOGITS,0.546344,0.874589,0.620917,0.889713,0.282688,0.818303,0.789207,0.990757


In [47]:
pd.DataFrame([mpnn_results], index=['MPNN'])

,IA,IAmix,IB,IBmix,IAB,PA,PB,PAB
MPNN,0.568086,0.957173,0.590632,0.899829,0.231451,0.854835,0.882645,0.813827
